## Projeto 3 de Modelagem e Simulação do Mundo Fisico - Pêndulo de Ondas 

In [10]:
#Bibliotecas que serão utilizadas ao longo do projeto
import numpy as np
import matplotlib.pyplot as plt
from math import *
from scipy.integrate import odeint

#Parâmetros
l0 = 0.24 # comprimento do fio (m)
m = 0.01122 # massa da esfera (kg)
ρ = 1.25 # densidade do ar (kg/m^3)
r = 0.007 # raio da esfera (m)
g = 9.80 # aceleração gravitacional (m/s^2)
Cd = 0.47 # coeficiente de arrasto  (adimensional)
A = pi * (r ** 2) #área da secção tranversal da esfera (m^2)
t_lista = np.arange(0, 10, 1e-3) # Lista de tempo
x_0 = -l0  # posição inicial do pêndulo no eixo x
y_0 = 0 # posição inicial em y
vx_0 = 0 # velocidade inicial em x
vy_0 = 0 # velocidade inicial em y

Implementando a função 'pendulo'.

In [11]:
# função pendulo que retorna as derivadas
#a função recebe uma lista de valores e um instante de tempo

def pendulo(a, t):
    #abre os valores da lista
    x = a[0]  #posição no eixo x
    y = a[1]  #posição no eixo y
    vx = a[2]  #velocidade no eixo x
    vy = a[3]  #velocidade no eixo y
    
    #velocidade do pêndulo
    v = sqrt(vx ** 2 + vy ** 2)
    # Cálculo do ângulo alfa (angulo entre a velocidade e a direção horizontal)
    if v > 0:
        cos_alpha = vx/v
        sen_alpha = vy/v
    else:
        sen_alpha = 0
        cos_alpha = 0
    #cálculo do angulo teta entre o fio e a direção vertical
    sen_theta = x/l0
    cos_theta = -y/l0
    
    #cálculo da força de arrasto
    Fa = ρ * A * Cd * (v ** 2) * 0.5
    #cálculo da tração
    T = m*v**2/l0 + m*g*cos_theta
    # Equações diferenciais
    dxdt = vx
    dydt = vy
    dvxdt = (-T * sen_theta - Fa * cos_alpha) / m
    dvydt = ( T * cos_theta - Fa * sen_alpha) / m - g
    #agrupa em uma lista
    dadt = [dxdt, dydt, dvxdt, dvydt]
    #retorna a lista de derivadas
    return dadt

Implementando o gráfico da trajetória.

In [12]:
a_0 = [x_0, y_0, vx_0, vy_0] # lista de condições iniciais
a_lista = odeint(pendulo, a_0, t_lista) # utilizando a função odeint obtemos a lista com os valores de x, y, vx e vy em cada instante

#abre os valores de a_lista
x_lista = a_lista[:,0] #posições do pêndulo no eixo x
y_lista = a_lista[:,1] #posições do pêndulo no eixo y
vx_lista = a_lista[:,2] #componente horizontal da velocidade
vy_lista = a_lista[:,3]  #componente vertical da velocidade

# Plota gráfico da trajetória do modelo
plt.plot(x_lista, y_lista, 'r')
plt.xlabel('x(m)')
plt.ylabel('y(m)')
plt.title('Gráfico da trajetória')
plt.grid(True)
plt.show()

In [13]:
# plota gráfico do alcance vertical em função do tempo
plt.plot(t_lista, y_lista)
plt.xlabel('Tempo(s)')
plt.ylabel('y(m)')
plt.title('Alcance vertical em função do tempo')
plt.grid(True)
plt.show()

Validação do Modelo Utilizando dados importados do tracker

In [14]:
#importa a biblioteca para obter os dados da validação
import pandas

# paramentros de posição real (o experimento foi realizado com o pêndulo partindo das posições abaixo)
x_0 = 0.15 # posição inicial do pendulo no eixo x (m)
y_0 = -0.104 # posição inicial do pendulo no eixo y(m)

l0 = (x_0**2+y_0**2)**(1/2) # comprimento do fio (m)
tt_lista = t_lista = np.arange(0, 5, 1e-3)  # lista de tempo para validação do modelo

a_0 = [x_0, y_0, vx_0, vy_0] # lista de condições iniciais
a_lista = odeint(pendulo, a_0, tt_lista) # lista utilizando odeint para validacao do modelo

# desmenbrando a lista a_lista
x = a_lista[:,0] #posição x
y = a_lista[:,1] #posiçao y
vx = a_lista[:,2] #velocidade x
vy = a_lista[:,3] #velocidade y

colunas = ['t', 'x', 'y', 'vx', 'vy'] # nome das colunas do arquivo csv
dados = pandas.read_csv('experimento.csv', names =  colunas) # importando os dados do experimento

#dados do experimento
te = dados.t.tolist() # tempo
xe = dados.x.tolist() # posicao em x
ye = dados.y.tolist() # posicao em y
vxe = dados.vx.tolist() # velocidade em x
vye = dados.vy.tolist() # velocidade em y

#define tempo de inicio do experimento
te0 = te[0]
for i in range(0,len(te)):
    #realizando ajustes de medida
    te[i] = te[i] - te0
    ye[i] = ye[i] + 0.02


Plotando os gráficos de validação

In [15]:

# plotando os graficos do modelo e do experimento da trajetória do pêndulo 
plt.plot(x, y, label =  'Modelo')
plt.plot(xe, ye, 'r', label = "Experimento")
plt.title("Trajetória do pêndulo")
plt.legend()
plt.xlabel("x (m)")
plt.ylabel("y (m)")
plt.axis('equal')
plt.grid()
plt.show()
    


In [16]:
# plotando os graficos do modelo e do experimento do alcance vertical do pendulo
plt.plot(tt_lista[0:2000], y[0:2000], label = 'Modelo')
plt.plot(te[0:480], ye[0:480], 'ro', label = 'Experimento')
plt.xlabel('tempo (s)')
plt.ylabel('y (m)')
plt.legend()
plt.title('Y em função do tempo')
plt.grid(True)
plt.show()

pares_exp = []
for i in range(len(te)):
    pares_exp.append([xe[i], ye[i]])
    
pares = []
for i in range(len(x)):
    pares.append(['{0:.3f}, {1:.3f}'.format(x[i], y[i])])


In [17]:
# redefinindo os parâmetros
l0 = 0.14 # (m)
m = 0.01122 # (kg)
ρ = 1 # (kg/m^3)
r = 0.007 # (m)
g = 9.80 # (m/s^2)
Cd = 0.47 # adimensional
A = pi * (r ** 2)
t_lista = np.arange(0, 10, 1e-3)
x_0 = -l0 # 0.154 -0.124
y_0 = 0
vx_0 = 0
vy_0 = 0

#criando uma nova função que recebe, também, o comprimento do fio como argumento
#pendulo2 recebe a lista de valores de x, y, vx e vy, um instante de tempo e o comprimento do fio, e retorna a lista de derivadas

def pendulo2(a, t, l0):
    #analogamente à função pendulo:
    x = a[0]
    y = a[1]
    vx = a[2]
    vy = a[3]
    v = sqrt(vx ** 2 + vy ** 2)
    if v > 0:
        cos_alpha = vx/v
        sen_alpha = vy/v
    else:
        sen_alpha = 0
        cos_alpha = 0
    sen_theta = x/l0
    cos_theta = -y/l0
    Fa = ρ * A * Cd * (v ** 2) * 0.5
    T = m*v**2/l0 + m*g*cos_theta
    dxdt = vx
    dydt = vy
    dvxdt = (-T * sen_theta - Fa * cos_alpha) / m
    dvydt = ( T * cos_theta - Fa * sen_alpha) / m - g
    dadt = [dxdt, dydt, dvxdt, dvydt]
    return dadt

comprimento_lista = [0.10, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20] #cria uma lista de comprimentos
altura_max_lista = [] #cria um alista para adicionar a altura maxima
periodo = [] #cria uma lista para adicionar os períodos

# calcula altura 
def calcula_altura(y_lista):
    for i in range(1,len(y_lista)): # percorre a lista de y
        if (y_lista[i] > y_lista[i-1]) and (y_lista[i] > y_lista[i+1]): # verifica se y[i] é maior que os dois vizinhos
            return y_lista[i] # retorna o valor de y[i] (y máximo)

# calcula periodo       
def calcula_periodo (y_lista, lista_t): 
    for i in range(1,len(y_lista)): #percorre a lista de y
        if (y_lista[i] > y_lista[i-1]) and (y_lista[i] > y_lista[i+1]): # verifica se y[i] é maior que os dois vizinhos
            return lista_t[i]  #retorna o instante em que o y maximo ocorreu

x_trajetoria = []
y_trajetoria = []
comprimento_lista = [0.10, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20] #cria uma lista de comprimentos

for l0 in comprimento_lista:
    a_0 = [-l0, y_0, vx_0, vy_0] #condições iniciais
    a_lista = odeint(pendulo2, a_0, t_lista, args=(l0,)) #resolve a função 
    #desmembra a_lista
    x_lista = a_lista[:,0] 
    y_lista = a_lista[:,1] 
    
    x_trajetoria.append(x_lista)
    y_trajetoria.append(y_lista)
    
    altura_max_lista.append(calcula_altura(y_lista))  # adiciona cada altura máxima à lista com as alturas máximas
    periodo.append(calcula_periodo(y_lista, t_lista))  # adiciona o momento da altura maxima à lista com os períodos
    
    plt.plot(t_lista, a_lista[:, 1], label=l0) # altura em função do tempo (gráfico intermediário)

#plota o grafico intermediário
plt.title("Variação do alcance em função do tempo, de acordo com o comprimento do pêndulo")
plt.xlabel("Tempo (s)")
plt.ylabel("Y (m)")
plt.grid(True)
plt.legend()
plt.show()

#plota o primeiro gráfico conclusivo, do alcance maximo em y em função do comprimento
plt.scatter(comprimento_lista, altura_max_lista) # plota o alcance máximo em função do comprimento
plt.plot(comprimento_lista, altura_max_lista)
plt.title("Variação do alcance máximo em função do comprimento do pêndulo")
plt.xlabel("Comprimento do pêndulo (m)")
plt.ylabel("Y (m)")
plt.grid(True)
plt.show()

#plota o grafico conclusivo do periodo em função do comprimento
plt.scatter(comprimento_lista, periodo) # plota o período em função do comprimento
plt.plot(comprimento_lista, periodo)
plt.title("Variação do período em função do comprimento do pêndulo")
plt.xlabel("Comprimento do pêndulo (m)")
plt.ylabel("Período (s)")
plt.grid(True)
plt.show()



In [18]:
# ANIMAÇÃO
%matplotlib qt5
plt.close('all')
get_ipython().magic('matplotlib qt5')

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython import display
import matplotlib.patches as patches
from matplotlib import animation

plt.rcParams['animation.ffmpeg_path'] = '/path_to_your/ffmpeg'

fig = plt.figure(figsize=(10,7))

ax = fig.add_subplot(111)
ax.set_xlim(-0.5, 0.5)
ax.set_ylim(-0.25, 0.05)


plt.title('Pêndulo de Ondas')
plt.xlabel('$x \quad [m]$')
plt.ylabel('$y \quad [m]$')
plt.axis('equal')

comprimento_lista = [0.10, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20] #cria uma lista de comprimentos

esfera1 = patches.Circle((0, comprimento_lista[0] ), r, fc='b')
esfera2 = patches.Circle((0, comprimento_lista[1]), r, fc='b')
esfera3 = patches.Circle((0, comprimento_lista[2]), r, fc='b')
esfera4 = patches.Circle((0, comprimento_lista[3]), r, fc='b')
esfera5 = patches.Circle((0, comprimento_lista[4]), r, fc='b')
esfera6 = patches.Circle((0, comprimento_lista[5]), r, fc='b')
esfera7 = patches.Circle((0, comprimento_lista[6]), r, fc='b')
esfera8 = patches.Circle((0, comprimento_lista[7]), r, fc='b')
esfera9 = patches.Circle((0, comprimento_lista[8]), r, fc='b')
esfera10 = patches.Circle((0, comprimento_lista[9]), r, fc='b')
esfera11 = patches.Circle((0, comprimento_lista[10]), r, fc='b')
 
elastico1, = ax.plot([],[])
elastico2, = ax.plot([],[])
elastico3, = ax.plot([],[])
elastico4, = ax.plot([],[])
elastico5, = ax.plot([],[])
elastico6, = ax.plot([],[])
elastico7, = ax.plot([],[])
elastico8, = ax.plot([],[])
elastico9, = ax.plot([],[])
elastico10, = ax.plot([],[])
elastico11, = ax.plot([],[])

def init():
    ax.add_patch(esfera1)
    ax.add_patch(esfera2)
    ax.add_patch(esfera3)
    ax.add_patch(esfera4)
    ax.add_patch(esfera5)
    ax.add_patch(esfera6)
    ax.add_patch(esfera7)
    ax.add_patch(esfera8)
    ax.add_patch(esfera9)
    ax.add_patch(esfera10)
    ax.add_patch(esfera11)
    elastico1.set_data([],[])
    elastico2.set_data([],[])
    elastico3.set_data([],[])
    elastico4.set_data([],[])
    elastico5.set_data([],[])
    elastico6.set_data([],[])
    elastico7.set_data([],[])
    elastico8.set_data([],[])
    elastico9.set_data([],[])
    elastico10.set_data([],[])
    elastico11.set_data([],[])
    return None

def animate(i):    
    esfera1.center = x_trajetoria[0][i] , y_trajetoria[0][i]
    elastico1.set_data([0,x_trajetoria[0][i]],[0,y_trajetoria[0][i]])
    esfera2.center = x_trajetoria[1][i] , y_trajetoria[1][i]
    elastico2.set_data([0,x_trajetoria[1][i]],[0,y_trajetoria[1][i]])
    esfera3.center = x_trajetoria[2][i] , y_trajetoria[2][i]
    elastico3.set_data([0,x_trajetoria[2][i]],[0,y_trajetoria[2][i]])
    esfera4.center = x_trajetoria[3][i] , y_trajetoria[3][i]
    elastico4.set_data([0,x_trajetoria[3][i]],[0,y_trajetoria[3][i]])
    esfera5.center = x_trajetoria[4][i] , y_trajetoria[4][i]
    elastico5.set_data([0,x_trajetoria[4][i]],[0,y_trajetoria[4][i]])
    esfera6.center = x_trajetoria[5][i] , y_trajetoria[5][i]
    elastico6.set_data([0,x_trajetoria[5][i]],[0,y_trajetoria[5][i]])
    esfera7.center = x_trajetoria[6][i] , y_trajetoria[6][i]
    elastico7.set_data([0,x_trajetoria[6][i]],[0,y_trajetoria[6][i]])
    esfera8.center = x_trajetoria[7][i] , y_trajetoria[7][i]
    elastico8.set_data([0,x_trajetoria[7][i]],[0,y_trajetoria[7][i]])
    esfera9.center = x_trajetoria[8][i] , y_trajetoria[8][i]
    elastico9.set_data([0,x_trajetoria[8][i]],[0,y_trajetoria[8][i]])
    esfera10.center = x_trajetoria[9][i] , y_trajetoria[9][i]
    elastico10.set_data([0,x_trajetoria[9][i]],[0,y_trajetoria[9][i]])
    esfera11.center = x_trajetoria[10][i] , y_trajetoria[10][i]
    elastico11.set_data([0,x_trajetoria[10][i]],[0,y_trajetoria[10][i]])
    return None

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=len(y_lista), interval=0.01, blit=False)


C:\Users\lucas\AppData\Local\Temp\ipykernel_3540\3366101675.py:4: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('matplotlib qt5')
